In [3]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.models as models
import time
import numpy as np
from torchvision import datasets, transforms
from tqdm import tqdm

ImportError: cannot import name 'PILLOW_VERSION' from 'PIL' (/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/PIL/__init__.py)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import platform
print(platform.platform())

macOS-12.3.1-arm64-arm-64bit


In [ ]:
CPU= False
device = "cpu" if CPU else torch.device("mps")
print("Device is :: {}".format(device))

Device is :: mps


In [ ]:
class CFG:
    lr = 0.001
    train_batch_size = 4
    total_epochs = 1
    num_classes = 10
    input_shape = (224,224)

In [ ]:
torch.manual_seed(1)
np.random.seed(1)

In [ ]:
image_path = "../data/"

In [ ]:
mnist_dataset = torchvision.datasets.MNIST(image_path,
                                'train',
                                download=False,
                                transform=transforms.Compose(
                                    [transforms.Resize(CFG.input_shape),
                                    transforms.Grayscale(3),ToTensor()]))

In [ ]:
trainset_1 = torch.utils.data.Subset(mnist_dataset, list(range(1000)))


In [ ]:
mnist_loader  = DataLoader(trainset_1,batch_size=CFG.train_batch_size,shuffle=True,num_workers=4)

In [ ]:
x_batch, y_batch = (next(iter(mnist_loader)))

In [ ]:
class MODELS:
    vgg16_model = models.vgg16(pretrained=True)
    alexnet = models.alexnet(pretrained=True)
    resnet_18 = models.resnet18(pretrained=True)
    mobilenet_v2 = models.mobilenet_v2(pretrained=True)
    efficientnet_b0 = models.efficientnet_b0(pretrained=True)
    squeezenet = models.squeezenet1_0(pretrained=True)

    vgg16_model.classifier[6] = nn.Linear(vgg16_model.classifier[6].in_features,CFG.num_classes)
    alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features,CFG.num_classes)
    resnet_18.fc = nn.Linear(resnet_18.fc.in_features,CFG.num_classes)
    mobilenet_v2.classifier[1] = nn.Linear(mobilenet_v2.classifier[1].in_features,CFG.num_classes)
    efficientnet_b0.classifier[1] = nn.Linear(efficientnet_b0.classifier[1].in_features,CFG.num_classes)
    squeezenet.classifier[1] = nn.Linear(squeezenet.classifier[1].in_channels,CFG.num_classes)

In [ ]:
def train(model_name,model,train_dl,n_epochs=CFG.total_epochs):
    '''
    call train_one_epoch:
    we will take average time taken to train per epoch for a max of 5 epochs
    '''
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=CFG.lr)
    average_time = []
    for epoch in range(n_epochs):
        start_time = time.time()
        print(f"Epoch {epoch} -->")
        pbar = tqdm(enumerate(train_dl), total=len(train_dl), desc='Train : '+model_name)
        for step, (x_batch, y_batch) in pbar:   
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            pred = model(x_batch)[:,0]
            loss = loss_fn(pred,y_batch.float())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        end_time = time.time() - start_time
        average_time.append(end_time)
    return np.mean(average_time)
    

In [ ]:
model_dict = {
    'vgg16_model' : MODELS.vgg16_model, 
    'alexnet' : MODELS.alexnet, 
    'resnet_18' : MODELS.resnet_18, 
    'mobilenet_v2' : MODELS.mobilenet_v2, 
    'efficientnet_b0' : MODELS.efficientnet_b0, 
    # 'squeezenet' : MODELS.squeezenet, 
}


In [ ]:
time_calc = {}
for model_name,model in model_dict.items():
    print("Model name is :: {}".format(model_name))
    print("-----------------------------------------")
    model_epoch_avg_time = train(model_name,model.to(device),mnist_loader)
    time_calc[model_name] = model_epoch_avg_time

Model name is :: vgg16_model
-----------------------------------------
Epoch 0 -->


Train : vgg16_model: 100%|██████████| 250/250 [16:50<00:00,  4.04s/it]   


Model name is :: alexnet
-----------------------------------------
Epoch 0 -->


Train : alexnet: 100%|██████████| 250/250 [01:38<00:00,  2.54it/s]

Model name is :: resnet_18
-----------------------------------------
Epoch 0 -->



Train : resnet_18: 100%|██████████| 250/250 [01:35<00:00,  2.61it/s]

Model name is :: mobilenet_v2
-----------------------------------------
Epoch 0 -->



Train : mobilenet_v2: 100%|██████████| 250/250 [3:19:32<00:00, 47.89s/it]    

Model name is :: efficientnet_b0
-----------------------------------------
Epoch 0 -->



Train : efficientnet_b0: 100%|██████████| 250/250 [1:27:45<00:00, 21.06s/it]   


In [ ]:
time_calc

{'vgg16_model': 1010.3419518470764,
 'alexnet': 98.45582103729248,
 'resnet_18': 95.7878258228302,
 'mobilenet_v2': 11972.771821022034,
 'efficientnet_b0': 5265.4243841171265}